In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/clase_11/clase_11/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Práctica adicional sobre joins con pandas


Trabajaremos con un dataset de campañas de marketing. El mismo está representado por un conjunto de tablas relacionales:

* train: relaciona id de consumidores, cupones de descuento, campaña de marketing y contiene la variable target 'redemption_status', que dice si el cupón fue canjeado o no.

* campaign data: fecha de inicio y finalización de la campaña y tipo de campaña

* customer_demographics: rango etareo, estado civil, tamaño de la familia, nro de hijos, nivel de ingresos, propietario/inquilino/hipoteca

* coupon_item_mapping: cada cupón da beneficios sobre distintos items

* item_data: marca, tipo de marca y categoría del item

* customer_transaction_data: datos sobre las compras hechas por los consumidores. No sabemos en qué compra usaron el cupón de descuento, tal vez la promoción sirve para más de una compra.

<img src='../Data/Schema.png'>

<b>Origen del dataset:</b>

https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [3]:
train=pd.read_csv('../Data/train.csv')
cust_dem=pd.read_csv('../Data/customer_demographics.csv')
trans=pd.read_csv('../Data/customer_transaction_data.csv')
coupon_item=pd.read_csv('../Data/coupon_item_mapping.csv')
item_data=pd.read_csv('../Data/item_data.csv')
campaign_data=pd.read_csv('../Data/campaign_data.csv')

---
#### Train dataframe

* Mirar el dataframe y chequear si hay valores nulos

In [4]:
display(train.head(3))
print('\nHay nulos:',train.isnull().any().any())

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0



Hay nulos: False


---
#### Customer demographics dataframe

* Chequear si hay nulos
* Mirar los dtypes
* Imputar los valores nulos por la moda de las columnas
* Llevar las columnas de variables categóricas a dtype 'category'

In [5]:
# cust_dem
display(cust_dem.head(3))
print('\n-------------------------')
print('Dtypes:\n') 
print(cust_dem.dtypes)
print('\n-------------------------')
print('Nulos:\n')      
print(cust_dem.isnull().sum())

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3



-------------------------
Dtypes:

customer_id        int64
age_range         object
marital_status    object
rented             int64
family_size       object
no_of_children    object
income_bracket     int64
dtype: object

-------------------------
Nulos:

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64


Imputamos valores faltantes por la moda

In [6]:
# Imputamos valores faltantes
imp=SimpleImputer(strategy='most_frequent')
cust_dem[['marital_status','no_of_children']]=imp.fit_transform(cust_dem[['marital_status','no_of_children']])
print('Nulos:\n')      
print(cust_dem.isnull().sum())

Nulos:

customer_id       0
age_range         0
marital_status    0
rented            0
family_size       0
no_of_children    0
income_bracket    0
dtype: int64


Llevamos variables a tipo categóricas

In [7]:
# categorical variables
cust_dem.iloc[:,1:]=cust_dem.iloc[:,1:].astype('category')
print('Dtypes:\n') 
print(cust_dem.dtypes)

Dtypes:

customer_id          int64
age_range         category
marital_status    category
rented            category
family_size       category
no_of_children    category
income_bracket    category
dtype: object


---
#### Transactions dataframe
* Chequear si hay nulos
* ver los dtypes
* dropear la columna 'dates'

In [8]:
# trans data
display(trans.head(3))
print('\nHay nulos:',trans.isnull().any().any())
print('\nDtypes:\n') 
print(trans.dtypes)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0



Hay nulos: False

Dtypes:

date                object
customer_id          int64
item_id              int64
quantity             int64
selling_price      float64
other_discount     float64
coupon_discount    float64
dtype: object


In [9]:
trans=trans.drop('date',axis=1)

In [10]:
trans.head(3)
print('\nDtypes:\n') 
print(trans.dtypes)


Dtypes:

customer_id          int64
item_id              int64
quantity             int64
selling_price      float64
other_discount     float64
coupon_discount    float64
dtype: object


---
#### Coupon-Item dataframe
* chequear si hay nulos

In [11]:
# Coupon item map
display(coupon_item.head(3))
print('\nHay nulos:',coupon_item.isnull().any().any())

,coupon_id,item_id
0,105,37
1,107,75
2,494,76



Hay nulos: False


---
#### Item dataframe
* Chequear si hay nulos
* Llevar las columnas categóricas a dtype 'category'

In [12]:
# Item Data
display(item_data.head(3))
item_data[['brand','brand_type','category']]=item_data[['brand','brand_type','category']].astype('category')
print(item_data.dtypes)
print('\nHay nulos:',item_data.isnull().any().any())

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery


item_id          int64
brand         category
brand_type    category
category      category
dtype: object

Hay nulos: False


---
#### Campaign dataframe
* Chequear si hay valores nulos
* Llevar las columnas start_date y end_date a tipo TimeStamp
* Calcular el período de las promos como end_date - start_date
* Dropear start_date y end_date

In [13]:
display(campaign_data.head(3))
print(campaign_data.dtypes)
print('\nHay nulos:',campaign_data.isnull().any().any())

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13


campaign_id       int64
campaign_type    object
start_date       object
end_date         object
dtype: object

Hay nulos: False


Calculamos cuanto tiempo duró la promoción

In [14]:
# Calculamos el período de la promoción

start_date=pd.to_datetime(campaign_data['start_date'],dayfirst=True)
end_date=pd.to_datetime(campaign_data['end_date'],dayfirst=True)

campaign_data['duration']=end_date-start_date # Periodo de validez del cupon
campaign_data['duration']=campaign_data['duration'].apply(lambda x: x.days)

campaign_data['campaign_type']=campaign_data['campaign_type'].astype('category')

campaign_data['month']=start_date.apply(lambda x : x.month).astype('category')
campaign_data['year']=start_date.apply(lambda x : x.year).astype('category')


In [15]:
campaign_data=campaign_data.drop(['start_date','end_date'],axis=1)

display(campaign_data.head(3))
print(campaign_data.dtypes)

,campaign_id,campaign_type,duration,month,year
0,24,Y,60,10,2013
1,25,Y,32,10,2013
2,20,Y,70,9,2013


campaign_id         int64
campaign_type    category
duration            int64
month            category
year             category
dtype: object


---
#### Merge Dataframes
1. Hacer un merge entre train y campaign_data. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
2. Hacer un merge de 1. con customer_demographics. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
3. Chequear valores nulos en 2.
4. Imputamos valores faltantes en las columnas demográficas. Para eso, sampleamos con reposición de la distribución conjunta de atributos demográficos observados.
5. Merge con las transacciones promedio por sujeto:
* Hacer un groupby por consumidor en el dataframe de transaccions
* Agregar el valor medio a las variables continuas ('selling_price','quantity', etc)
* merge con el dataframe obtenido en 4. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
6. Merge con transacciones promedio por item
* a. Hacer un groupby por item en el dataframe de transaccions
* b. Calcular el valor medio de las variables continuas
* c. Hacer un merge con el dataframe 'coupon_item'
* d. Hacer un merge de c. con item_data ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
* e. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la moda de las variables categóricas 'brand','brand_type,'category'
* f. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la media de las variables continuas.
* g. Hacer un merge de d. con e. y luego con f. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
7. Guardar el dataframe resultante en un csv para la práctica de la clase.

In [16]:
train.shape,campaign_data.shape, cust_dem.shape,trans.shape,coupon_item.shape,item_data.shape

((78369, 5), (28, 5), (760, 7), (1324566, 6), (92663, 2), (74066, 4))

<b>Hacemos un merge entre train y campaign data

In [17]:
data=pd.merge(train,campaign_data,on='campaign_id',how='left')
print(data.shape)
data.isnull().sum()


(78369, 9)


id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
campaign_type        0
duration             0
month                0
year                 0
dtype: int64

<b> Merge con los datos demográficos de los consumidores

In [18]:
data=pd.merge(data,cust_dem,on='customer_id',how='left')
data.shape

(78369, 15)

In [19]:
data.isnull().sum()

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
campaign_type            0
duration                 0
month                    0
year                     0
age_range            34708
marital_status       34708
rented               34708
family_size          34708
no_of_children       34708
income_bracket       34708
dtype: int64

<b> Imputamos valores faltentes</b>

Vamos a samplear de la distribución conjunta de variables con valores faltantes. De esta manera conservamos la variabilidad de los datos (no imputamos todo por la moda) y la estructura de correlaciones entre variables (no asignamos un valor de age_range incompatible con uno el de family_size)

In [20]:
# Imputamos valores faltantes sampleando con reposicion de los datos observados
# De esta manera preservamos la estructura de correlaciones observada entre las columnas demográficas

impute_columns=['age_range','marital_status','rented','family_size','no_of_children','income_bracket']

null_mask=data[impute_columns].isnull().all(axis=1)
random_samples=data.loc[~null_mask,impute_columns].sample(n=null_mask.sum(),replace=True)
data.loc[null_mask,impute_columns]=random_samples.values

data.isnull().sum()

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
campaign_type        0
duration             0
month                0
year                 0
age_range            0
marital_status       0
rented               0
family_size          0
no_of_children       0
income_bracket       0
dtype: int64

<b> Merge con información de transacciones promedio por cliente</b>

Agregamos la información de la actividad de compras típicas hechas por cada consumidor.

In [21]:
# Merge transactions: calculo la media de los valores de las transacciones por sujeto
trans_customer=trans.groupby('customer_id')[['quantity','selling_price','other_discount','coupon_discount']].agg('mean')
trans_customer.columns=['mean_'+col+'_cust' for col in trans_customer.columns]
trans_customer=trans_customer.reset_index();

In [35]:
trans_customer.head(3)

,customer_id,mean_quantity_cust,mean_selling_price_cust,mean_other_discount_cust,mean_coupon_discount_cust
0,1,1.170802,94.001842,-16.250382,-2.019876
1,2,1.131265,102.864033,-16.830430,-0.595084
2,3,11.578723,103.617404,-22.714227,-3.091546


In [22]:
data=pd.merge(data,trans_customer,on='customer_id',how='left')

In [23]:
data.shape

(78369, 19)

In [24]:
data.head(3)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,duration,month,year,age_range,marital_status,rented,family_size,no_of_children,income_bracket,mean_quantity_cust,mean_selling_price_cust,mean_other_discount_cust,mean_coupon_discount_cust
0,1,13,27,1053,0,X,47,5,2013,46-55,Married,0,1,1,5,340.487097,184.260484,-33.168935,-0.287258
1,2,13,116,48,0,X,47,5,2013,36-45,Married,0,2,1,3,31.540260,234.247013,-27.699169,-3.215039
2,6,9,635,205,0,Y,32,3,2013,46-55,Married,0,2,1,7,1.392784,121.094495,-17.795660,-2.212082


<b> Transacciones promedio por item </b>

Tenemos el mapeo entre cupón e item. Vamos a agregar información sobre cuánto se consume cada item.

In [25]:
# Merge with item data
trans_item=trans.groupby('item_id')[['quantity','selling_price','other_discount','coupon_discount']].mean()
trans_item.columns=['mean_'+col+'_item' for col in trans_item.columns]
trans_item=trans_item.reset_index()
trans_item.head(3)


,item_id,mean_quantity_item,mean_selling_price_item,mean_other_discount_item,mean_coupon_discount_item
0,1,1.0,124.31,0.0,0.0
1,2,1.0,35.26,0.0,0.0
2,3,1.0,56.64,0.0,0.0


In [26]:
coupon_item_trans=pd.merge(coupon_item,trans_item,on='item_id',how='inner')
coupon_item_trans_brand=pd.merge(coupon_item_trans,item_data,on='item_id',how='left')
coupon_item_trans_brand.head(3)

,coupon_id,item_id,mean_quantity_item,mean_selling_price_item,mean_other_discount_item,mean_coupon_discount_item,brand,brand_type,category
0,105,37,2.285714,113.12,-17.557143,0.0,56,Local,Grocery
1,6,37,2.285714,113.12,-17.557143,0.0,56,Local,Grocery
2,22,37,2.285714,113.12,-17.557143,0.0,56,Local,Grocery


In [27]:
coupon_item_trans_brand.dtypes

coupon_id                       int64
item_id                         int64
mean_quantity_item            float64
mean_selling_price_item       float64
mean_other_discount_item      float64
mean_coupon_discount_item     float64
brand                        category
brand_type                   category
category                     category
dtype: object

In [28]:
gb=coupon_item_trans_brand.groupby('coupon_id')
coupon_category=gb['category'].apply(lambda x:x.value_counts().index[0])
coupon_brand_type=gb['brand_type'].apply(lambda x:x.value_counts().index[0])
coupon_brand=gb['brand'].apply(lambda x:x.value_counts().index[0])

coupon_brand=pd.concat([pd.DataFrame(coupon_brand),pd.DataFrame(coupon_brand_type),pd.DataFrame(coupon_category)], axis=1).reset_index()
coupon_brand.head(3)

,coupon_id,brand,brand_type,category
0,1,1475,Established,Natural Products
1,2,2084,Established,Grocery
2,3,278,Established,Grocery


In [29]:
coupon_item_sales=coupon_item_trans_brand.groupby('coupon_id')[['mean_quantity_item','mean_selling_price_item','mean_other_discount_item','mean_coupon_discount_item']].mean()
coupon_item_sales.columns=[col+'_coupon' for col in coupon_item_sales]
coupon_item_sales=coupon_item_sales.reset_index()
coupon_item_sales.head(3)


,coupon_id,mean_quantity_item_coupon,mean_selling_price_item_coupon,mean_other_discount_item_coupon,mean_coupon_discount_item_coupon
0,1,1.221644,100.195701,-25.273218,-1.321524
1,2,1.137500,114.354438,-23.314062,0.000000
2,3,1.121525,129.981640,-17.543797,-2.747358


In [30]:
data=pd.merge(data,coupon_item_sales,on='coupon_id',how='left')

In [31]:
data=pd.merge(data,coupon_brand,on='coupon_id',how='left')

In [32]:
data[['brand','brand_type','category']]=data[['brand','brand_type','category']].astype('category')
data.dtypes

id                                     int64
campaign_id                            int64
coupon_id                              int64
customer_id                            int64
redemption_status                      int64
campaign_type                       category
duration                               int64
month                               category
year                                category
age_range                           category
marital_status                      category
rented                              category
family_size                         category
no_of_children                      category
income_bracket                      category
mean_quantity_cust                   float64
mean_selling_price_cust              float64
mean_other_discount_cust             float64
mean_coupon_discount_cust            float64
mean_quantity_item_coupon            float64
mean_selling_price_item_coupon       float64
mean_other_discount_item_coupon      float64
mean_coupo

In [33]:
data.shape

(78369, 26)

In [34]:
# Guardamos los datos
data.to_csv('../Data/data_preprocessed.csv')